### Imports

In [ ]:
import utils
import numpy as np
import seaborn as sns
from typing import List
import math
import csv
import pandas as pd

from tqdm import tqdm

### Functions

In [ ]:
def rat2vec(path: str) -> List[float]:
    vector = []
    with open(path, 'r') as f:
        for r in f:
            vector.append(float(r))
    return vector

# Pre {len(r1) = len(r2)}
def dist(v: List[float], w: List[float]) -> float:
    res = 0
    for i in range(len(v)):
        res += (v[i]-w[i])**2
    return math.sqrt(res)

def games2cmm(path: str) -> List[float]:
    with open(path, 'r') as f:
        reader = csv.reader(f, delimiter=' ')
        team_count = int(next(reader)[0])
        
        b = np.zeros((1,team_count))[0]
        c = np.zeros(shape=(team_count, team_count))
        
        for row in reader:
            _, team1, goals1, team2, goals2 = [int(x) for x in row]
            c[team1-1][team2-1]-=1
            c[team2-1][team1-1]-=1
            c[team1-1][team1-1]+=1
            c[team2-1][team2-1]+=1
            if (goals1 > goals2):
                b[team1-1] += 1
                b[team2-1] -= 1
            else:
                b[team1-1] -= 1
                b[team2-1] += 1
                
        for i in range(0,team_count):
            c[i][i] += 2
            b[i] /= 2
            b[i] += 1
        
        return np.linalg.solve(c,b).tolist()

### Run

In [ ]:
EX = ".expected"
IN = ".in"
OUT = ".out"
CMM = '0'

DIR_TEST = "../src/tests/test_completos/"
DIR_RESULT = "../data/"

tests = [
    "test_completo_1000_2",
    "test_completo_1000_8",
    "test_completo_100_4",
    "test_completo_100_8",
    "test_completo_10_1",    
]

In [ ]:
df = pd.DataFrame(columns=["method", "difference"])

for test in tqdm(tests):
    utils.run(path= DIR_TEST+test+IN, method=CMM, out= DIR_RESULT+test+OUT)
    cmm = rat2vec(DIR_RESULT+test+OUT) 
    cat = rat2vec(DIR_TEST+test+EX)
    lin = games2cmm(DIR_TEST+test+IN)
    
    df = df.append({
        "method" : "Nuestro",
        "difference" : dist(cmm, lin)
    }, ignore_index=True)
    
    df = df.append({
        "method" : "Cátedra",
        "difference" : dist(cat, lin)
    }, ignore_index=True)

In [ ]:
df

In [ ]:
sns.boxplot(data=df, x="method", y="difference")

In [ ]:
utils.run(path="../src/tests/test1.in", method='0', out="../data/test1.out")

u = rat2vec("../data/test1.out")
v = rat2vec("../src/tests/test1.expected")
w = games2cmm("../src/tests/test1.in")

print("Catedra:",dist(v,w))
print("Elim-Gaussiana:",dist(u,w))

In [ ]:
games2cmm("../src/tests/test2.in")